# LMP Data Generation for AGVis

Reference:

1. Zhang, Q., Li, F. A Dataset for Electricity Market Studies on Western and Northeastern Power Grids in the United States. Sci Data 10, 646 (2023). https://doi.org/10.1038/s41597-023-02448-w
1. https://github.com/enliten/ENLITEN-Grid-Econ-Data/

In [1]:
from tqdm import tqdm

import numpy as np
import pandas as pd
import scipy

import andes
import ams

import pypower.api as pyp

import datetime

import matplotlib
import matplotlib.pyplot as plt

In [2]:
matplotlib.rcdefaults()

In [3]:
%matplotlib inline

In [4]:
print("Last run time:", datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

print(f'andes:{andes.__version__}')
print(f'ams:{ams.__version__}')

Last run time: 2024-04-03 07:44:27
andes:1.9.1.post24+g7a87ad5d
ams:0.9.5.post1+g114d11e


In [5]:
andes.config_logger(stream_level=20)

In [6]:
sa = andes.load('./data/npcc_agvis.xlsx',
                setup=True,
                no_output=True,
                default_config=True,)

sa.PFlow.run()

sa.TDS.config.criteria = 0
sa.TDS.config.no_tqdm = True
sa.TDS.config.tf = 0.5

sa.TDS.run()

Working directory: "/Users/jinningwang/Documents/work/psal/src/notes/lmp"
> Loaded generated Python code in "/Users/jinningwang/.andes/pycode".
Parsing input file "./data/npcc_agvis.xlsx"...
Input file parsed in 0.4379 seconds.
System internal structure set up in 0.0359 seconds.
-> System connectivity check results:
  No islanded bus detected.
  System is interconnected.
  Each island has a slack bus correctly defined and enabled.

-> Power flow calculation
           Numba: Off
   Sparse solver: KLU
 Solution method: NR method
Power flow initialized in 0.0033 seconds.
0: |F(x)| = 0.004215993603
1: |F(x)| = 3.030635298e-08
Converged in 2 iterations in 0.0058 seconds.
GENCLS (vf range) out of typical lower limit.

   idx     | values | limit
-----------+--------+------
 GENCLS_3  | 0.990  | 1    
 GENCLS_4  | 0.926  | 1    
 GENCLS_16 | 0.933  | 1    


Initialization for dynamics completed in 0.0379 seconds.
Initialization was successful.
Simulation to t=0.50 sec completed in 0.0242 se

True

In [7]:
npcc_lmp_src = scipy.io.loadmat('./data/NPCC_LMP.mat')
npcc_lmp_data = npcc_lmp_src['LMP_c']

In [8]:
total_hour = 24 * 366  # 2020 is a leap year

# desired output hours
rows = np.arange(4488, 4644, 1)
total_hour = len(rows)

cols_vBus = [f'v Bus {str(i)}' for i in sa.Bus.idx.v]
cols = ['Time [s]'] + sa.dae.xy_name
npcc_out = pd.DataFrame(data=np.zeros((total_hour, len(cols))),
                        columns=cols)
npcc_out['Time [s]'] = np.arange(0, total_hour) / 100  # speed up the time

lmp_raw = npcc_lmp_data.T[rows, :]
lmp2 = (lmp_raw - lmp_raw.mean(axis=0)) / lmp_raw.std(axis=0)
npcc_out[cols_vBus] = lmp2

In [9]:
npcc_out[cols_vBus].describe().round(2)

,v Bus 1,v Bus 2,v Bus 3,v Bus 4,v Bus 5,v Bus 6,v Bus 7,v Bus 8,v Bus 9,v Bus 10,...,v Bus 131,v Bus 132,v Bus 133,v Bus 134,v Bus 135,v Bus 136,v Bus 137,v Bus 138,v Bus 139,v Bus 140
count,156.00,156.00,156.00,156.00,156.00,156.00,156.00,156.00,156.00,156.00,...,156.00,156.00,156.00,156.00,156.00,156.00,156.00,156.00,156.00,156.00
mean,-0.00,-0.00,-0.00,0.00,-0.00,-0.00,-0.00,0.00,-0.00,-0.00,...,0.00,-0.00,0.00,-0.00,0.00,-0.00,-0.00,-0.00,0.00,0.00
std,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,...,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00
min,-0.26,-0.26,-0.26,-0.26,-0.26,-0.26,-0.26,-0.26,-0.26,-0.26,...,-4.04,-4.04,-4.04,-0.31,-4.04,-3.61,-1.00,-0.27,-4.04,-0.42
25%,-0.26,-0.26,-0.26,-0.26,-0.26,-0.26,-0.26,-0.26,-0.26,-0.26,...,0.24,0.23,0.25,-0.30,0.25,-0.21,-0.85,-0.27,0.25,-0.29
50%,-0.25,-0.25,-0.25,-0.25,-0.25,-0.25,-0.25,-0.25,-0.25,-0.25,...,0.25,0.23,0.25,-0.24,0.25,-0.07,-0.09,-0.25,0.25,-0.21
75%,-0.24,-0.24,-0.24,-0.24,-0.24,-0.24,-0.24,-0.24,-0.24,-0.24,...,0.25,0.26,0.25,-0.20,0.25,0.53,0.39,-0.23,0.25,-0.20
max,4.04,4.04,4.04,4.04,4.04,4.04,4.04,4.04,4.04,4.04,...,0.25,0.32,0.25,4.03,0.25,1.72,2.48,4.04,0.25,4.04


In [10]:
# export to ADNES output CSV file
npcc_out.to_csv('./data/npcc_lmp.csv', index=False)